# ⬇️ Instalación y carga del modelo

In [1]:
%%capture
!pip install -U transformers
!pip install accelerate
!pip install -U bitsandbytes
!pip install Wikipedia-API

In [2]:
import wikipediaapi

# Initializate Wikipedia object
wikipedia = wikipediaapi.Wikipedia(user_agent='QuestionGeneration (francoartico0@gmail.com)', language='es')

def clean_wikipedia_article(title, sections_to_remove):
    """
    Fetches a Wikipedia article and removes specified sections
    """
    page = wikipedia.page(title)

    if page.exists():
      for section in sections_to_remove:
          section_to_remove = page.section_by_title(section)
          if section_to_remove:
            page.sections.remove(section_to_remove)
      return page.text
    return None


In [3]:
sections_to_remove =["Enlaces externos", "Bibliografía", "Referencias", "Notas", "Véase también", "Notas y referencias", "Otras lecturas"]
article = clean_wikipedia_article(title="Bill Gates", sections_to_remove=sections_to_remove)

print(article)

William Henry Gates III (Seattle, 28 de octubre de 1955), conocido como Bill Gates, es un magnate empresarial, desarrollador de software, inversor, autor y filántropo estadounidense. Es cofundador de Microsoft, junto con su difunto amigo de la infancia Paul Allen. Durante su carrera en Microsoft, Gates ocupó los cargos de presidente, director ejecutivo (CEO), presidente y arquitecto jefe de software, además de ser el mayor accionista individual hasta mayo de 2014. Fue uno de los principales empresarios de la revolución de las microcomputadoras de las décadas de 1970 y 1980.
Su fortuna se calcula en 129 000 millones de dólares (2023) según la revista Forbes, hecho que lo colocó como el cuarto hombre más rico del mundo.[3]​ Antes del estallido de la burbuja de las punto com, su patrimonio neto ascendió a 114 100 millones de dólares, lo que lo convirtió en la décima persona más rica en toda la historia de la humanidad.[4]​
Es uno de los empresarios más conocidos que surgieron durante los 

In [4]:
import torch
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer

access_token = userdata.get('HUGGINGFACE_ACCESS_TOKEN')

checkpoint = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, token=access_token)
model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             device_map="auto",
                                             token=access_token,
                                             torch_dtype=torch.float16,
                                             quantization_config={"load_in_4bit": True}

)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
def generate_response(messages, tokenizer):
    inputs = tokenizer.apply_chat_template(messages,
                                           tokenize=True,
                                           add_generation_prompt=True,
                                           return_tensors="pt").to("cuda")

    print(f"Number of inputs tokens: {len(inputs[0])}")

    text_streamer = TextStreamer(tokenizer, skip_prompt=True) # real-time output, more memory-efficient
    _ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=1024, use_cache=True)

# ✅ Generación de preguntas con `Llama-3.1-8B-Instruct` usando zero-shot

In [6]:
messages = [
    {
        "role": "assistant",
        "content": (
            "Eres un generador de preguntas de lectura comprensiva. "
            "Tu tarea es crear preguntas basadas en un texto dado, "
            "sin incluir respuestas ni opciones. Solo debes generar preguntas, "
            "en las cuales deberás incluir preguntas de tipo verdadero o falso."
        ),
    },
    {
        "role": "user",
        "content": (
            "Genera exactamente 10 preguntas, de las cuales 5 deben ser de tipo "
            "verdadero o falso, sin incluir las respuestas ni opciones, sobre el siguiente texto:\n\n"
            f"{article}"
        ),
    },
]

generate_response(messages, tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Number of inputs tokens: 3067


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Aquí te dejo 10 preguntas sobre el texto, de las cuales 5 son de tipo verdadero o falso:

1. ¿En qué año nació Bill Gates? 
2. ¿Cuál es la fortuna de Bill Gates según la revista Forbes? 
3. ¿Cuál es el cargo que ocupó Bill Gates en Microsoft? 
4. ¿Cuándo se casó Bill Gates con Melinda French? 
5. ¿Cuál es el título de la serie documental que se lanzó sobre la vida de Bill Gates? 

Y estas 5 preguntas son de tipo verdadero o falso:

1. Es cierto que Bill Gates se casó con Melinda French en 1993. 
2. Falso que Bill Gates se retiró de Microsoft en 2008. 
3. Es cierto que Bill Gates es el cuarto hombre más rico del mundo. 
4. Falso que Bill Gates abandonó su matrimonio con Melinda French en 2020. 
5. Es cierto que Bill Gates es el fundador de Microsoft.<|eot_id|>


# 💥 Generación de preguntas con `Llama-3.1-8B-Instruct` usando few-shot

In [7]:
# Clean Wikipedia articles
turing = clean_wikipedia_article(title="Alan Turing", sections_to_remove=sections_to_remove)
von_neumann = clean_wikipedia_article(title="John von Neumann", sections_to_remove=sections_to_remove)

# Example questions for Alan Turing
turing_question_examples = """
Preguntas de tipo respuestas cortas:
1. ¿En qué fecha nació Alan Turing?
2. ¿Dónde nació Alan Turing?
3. ¿Cuáles eran las profesiones de Alan Turing?
4. ¿Cuál fue la causa de su muerte?
5. ¿A qué se dedicó durante la Segunda Guerra Mundial?

Preguntas de tipo Verdadero o Falso:
6. La máquina sobre la cual trabajó durante la guerra se llamó máquina Enigma. ¿Verdadero o Falso?
7. Se estima que el trabajo de Turing durante la guerra acortó la misma entre 2 y 4 años. ¿Verdadero o Falso?
8. La prueba de Turing se utiliza para juzgar la computabilidad de una función. ¿Verdadero o Falso?
9. Alan Turing fue condenado por crímenes de guerra después de la Segunda Guerra Mundial. ¿Verdadero o Falso?
"""

# Example questions for John von Neumann
von_neumann_question_examples = """
Preguntas de tipo respuestas cortas:
1. ¿A qué se dedicaba John von Neumann?
2. ¿Cuál fue la fecha de su nacimiento?
3. ¿En qué universidad obtuvo su doctorado en matemáticas?
4. ¿Cuál fue su aporte en la Segunda Guerra Mundial?

Preguntas de tipo Verdadero o Falso:
5. Es uno de los matemáticos más importantes del siglo XX. ¿Verdadero o Falso?
6. Asistió a la Universidad de Budapest. ¿Verdadero o Falso?
7. John von Neumann participó en el famoso Proyecto Manhattan. ¿Verdadero o Falso?
"""

# Define messages for the model
messages = [
    {
        "role": "assistant",
        "content": (
            "Eres un generador de preguntas de lectura comprensiva. "
            "Tu tarea es crear preguntas basadas en un texto dado, "
            "sin incluir respuestas ni opciones. Solo debes generar preguntas, "
            "incluyendo preguntas de tipo verdadero o falso."
        ),
    },
    {
        "role": "user",
        "content": (
            f"Usando el siguiente artículo {turing}, estos son ejemplos de preguntas que puedes hacer:\n"
            f"{turing_question_examples}\n\n"
            f"Usando el siguiente artículo {von_neumann}, estos son ejemplos de preguntas que puedes hacer:\n"
            f"{von_neumann_question_examples}\n\n"
            f"Genera exactamente 10 preguntas, incluyendo 5 que deben ser de tipo verdadero o falso, sobre el siguiente texto:\n{article}"
        ),
    },
]

# Generate response
generate_response(messages, tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Number of inputs tokens: 19768
Aquí te dejo 10 preguntas sobre el texto, incluyendo 5 preguntas de tipo "verdadero o falso".

**Preguntas de tipo respuestas cortas:**

1. ¿Cuál fue la fecha de nacimiento de Bill Gates?
2. ¿Cuál es el nombre de la fundación que fundó Bill Gates con su esposa Melinda?
3. ¿Cuál es el nombre del sistema operativo que creó Bill Gates y su empresa Microsoft?
4. ¿Cuál es el nombre del manuscrito de Leonardo da Vinci que adquirió Bill Gates?
5. ¿Cuál es el nombre del premio que recibió Bill Gates en 2006?

**Preguntas de tipo "verdadero o falso":**

1. Bill Gates es el fundador de Microsoft. ¿Verdadero o falso?
2. Bill Gates es el presidente de la Fundación Bill y Melinda Gates. ¿Verdadero o falso?
3. Bill Gates es el creador del sistema operativo Windows. ¿Verdadero o falso?
4. Bill Gates ha sido nombrado como uno de los "50 solteros más codiciados" por Good Housekeeping. ¿Verdadero o falso?
5. Bill Gates ha sido galardonado con el Premio Príncipe de Asturias